In [1]:
from Process import PlateReader
import pandas as pd

# Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [4]:
sample_info = pd.read_csv(r"03_16_21 Experiment Synthesis Info")
sample_info

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,dppc-ethanol-stock uL,dspepeg2000-ethanol-stock uL,pfh-ethanol-stock uL,ethanol-stock uL,water-stock uL
0,S2_A1_03-16-2021_Right,0.0001,0.000035,0.0,0.333333,0.666532,A1,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,336.064013,999.797500
1,S2_A2_03-16-2021_Right,0.0001,0.000035,0.0,0.444444,0.555421,A2,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,479.740702,833.130833
2,S2_A3_03-16-2021_Right,0.0001,0.000035,0.0,0.555556,0.444309,A3,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,614.311941,666.464167
3,S2_A4_03-16-2021_Right,0.0001,0.000035,0.0,0.666667,0.333198,A4,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,738.832881,499.797500
4,S2_A5_03-16-2021_Right,0.0001,0.000035,0.0,0.777778,0.222087,A5,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,62.377473,0.0,852.745468,333.130833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,S3_B4_03-16-2021_Right,0.0000,0.000000,0.0,0.444000,0.556000,B4,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,0.000000,0.0,616.560881,834.000000
60,S3_B5_03-16-2021_Right,0.0000,0.000000,0.0,0.556000,0.444000,B5,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,0.000000,0.0,752.208042,666.000000
61,S3_B6_03-16-2021_Right,0.0000,0.000000,0.0,0.667000,0.333000,B6,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,0.000000,0.0,876.568121,499.500000
62,S3_B7_03-16-2021_Right,0.0000,0.000000,0.0,0.778000,0.222000,B7,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,0.000000,0.0,990.340020,333.000000


# Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
        
        
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.


* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

*Hmm maybe make dataframes as dictionary so you can look and debug later*

In [5]:
path = r"C:\Users\Edwin\Downloads\03_16_21_Ouzo.xlsx"
plate_names = ['S2']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

In [8]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)

# Trim if needed
merged_df = merged_df[:len(sample_info)+1]
len(merged_df)

65

***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [9]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30) # should have something to detect OVERFLW and call it out
# merged_df.drop(['Wavelength'], inplace = True)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
Wavelength,200.000,210.000,220.000,230.000,240.000,250.000,260.000,270.000,280.000,290.000,...,900.000,910.000,920.000,930.000,940.000,950.000,960.000,970.000,980.000,990.000
A1,3.376,3.398,3.418,3.000,3.413,3.470,3.423,3.240,2.599,1.056,...,0.112,0.113,0.121,0.121,0.140,0.161,0.192,0.207,0.220,0.200
A2,3.337,3.370,3.386,3.664,3.406,3.459,3.416,3.225,2.562,0.994,...,0.070,0.072,0.072,0.078,0.087,0.108,0.134,0.148,0.154,0.158
A3,3.345,3.373,3.400,3.691,3.423,3.487,3.441,3.242,2.566,0.963,...,0.067,0.069,0.068,0.073,0.081,0.098,0.121,0.134,0.141,0.145
A4,3.341,3.369,3.391,3.683,3.428,3.476,3.439,3.236,2.539,0.987,...,0.068,0.071,0.068,0.072,0.078,0.092,0.111,0.123,0.129,0.135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E12,3.385,3.412,3.432,3.698,3.445,3.493,3.445,3.254,2.548,0.982,...,0.121,0.124,0.125,0.132,0.143,0.166,0.194,0.209,0.218,0.222
F1,3.319,3.351,3.366,3.642,3.388,3.439,3.386,3.203,2.475,0.907,...,0.076,0.078,0.075,0.082,0.090,0.110,0.134,0.148,0.154,0.163
F2,3.346,3.374,3.398,3.715,3.432,3.501,3.456,3.241,2.495,0.900,...,0.067,0.070,0.067,0.070,0.076,0.090,0.108,0.120,0.127,0.133
F3,3.341,3.376,3.401,3.719,3.440,3.509,3.462,3.246,2.587,0.952,...,0.066,0.070,0.065,0.067,0.071,0.080,0.093,0.103,0.109,0.116


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [10]:
platereader_df = PlateReader.merge_sampleinfo_platereader(sample_info, merged_df)
platereader_df

,200.0nm,210.0nm,220.0nm,230.0nm,240.0nm,250.0nm,260.0nm,270.0nm,280.0nm,290.0nm,...,900.0nm,910.0nm,920.0nm,930.0nm,940.0nm,950.0nm,960.0nm,970.0nm,980.0nm,990.0nm
A1,3.376,3.398,3.418,3.000,3.413,3.470,3.423,3.240,2.599,1.056,...,0.112,0.113,0.121,0.121,0.140,0.161,0.192,0.207,0.220,0.200
A2,3.337,3.370,3.386,3.664,3.406,3.459,3.416,3.225,2.562,0.994,...,0.070,0.072,0.072,0.078,0.087,0.108,0.134,0.148,0.154,0.158
A3,3.345,3.373,3.400,3.691,3.423,3.487,3.441,3.242,2.566,0.963,...,0.067,0.069,0.068,0.073,0.081,0.098,0.121,0.134,0.141,0.145
A4,3.341,3.369,3.391,3.683,3.428,3.476,3.439,3.236,2.539,0.987,...,0.068,0.071,0.068,0.072,0.078,0.092,0.111,0.123,0.129,0.135
A5,3.336,3.367,3.385,3.671,3.405,3.465,3.429,3.228,2.666,1.109,...,0.069,0.073,0.067,0.070,0.074,0.084,0.097,0.107,0.114,0.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E12,3.385,3.412,3.432,3.698,3.445,3.493,3.445,3.254,2.548,0.982,...,0.121,0.124,0.125,0.132,0.143,0.166,0.194,0.209,0.218,0.222
F1,3.319,3.351,3.366,3.642,3.388,3.439,3.386,3.203,2.475,0.907,...,0.076,0.078,0.075,0.082,0.090,0.110,0.134,0.148,0.154,0.163
F2,3.346,3.374,3.398,3.715,3.432,3.501,3.456,3.241,2.495,0.900,...,0.067,0.070,0.067,0.070,0.076,0.090,0.108,0.120,0.127,0.133
F3,3.341,3.376,3.401,3.719,3.440,3.509,3.462,3.246,2.587,0.952,...,0.066,0.070,0.065,0.067,0.071,0.080,0.093,0.103,0.109,0.116


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [11]:
sample_info.reset_index(drop=True, inplace=True)
platereader_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, platereader_df], axis = 1)
combined_df

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,dppc-ethanol-stock uL,...,900.0nm,910.0nm,920.0nm,930.0nm,940.0nm,950.0nm,960.0nm,970.0nm,980.0nm,990.0nm
0,S2_A1_03-16-2021_Right,0.0001,0.000035,0.0,0.333333,0.666532,A1,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.112,0.113,0.121,0.121,0.140,0.161,0.192,0.207,0.220,0.200
1,S2_A2_03-16-2021_Right,0.0001,0.000035,0.0,0.444444,0.555421,A2,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.070,0.072,0.072,0.078,0.087,0.108,0.134,0.148,0.154,0.158
2,S2_A3_03-16-2021_Right,0.0001,0.000035,0.0,0.555556,0.444309,A3,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.067,0.069,0.068,0.073,0.081,0.098,0.121,0.134,0.141,0.145
3,S2_A4_03-16-2021_Right,0.0001,0.000035,0.0,0.666667,0.333198,A4,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.068,0.071,0.068,0.072,0.078,0.092,0.111,0.123,0.129,0.135
4,S2_A5_03-16-2021_Right,0.0001,0.000035,0.0,0.777778,0.222087,A5,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.069,0.073,0.067,0.070,0.074,0.084,0.097,0.107,0.114,0.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,S3_B4_03-16-2021_Right,0.0000,0.000000,0.0,0.444000,0.556000,B4,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.121,0.124,0.125,0.132,0.143,0.166,0.194,0.209,0.218,0.222
60,S3_B5_03-16-2021_Right,0.0000,0.000000,0.0,0.556000,0.444000,B5,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.076,0.078,0.075,0.082,0.090,0.110,0.134,0.148,0.154,0.163
61,S3_B6_03-16-2021_Right,0.0000,0.000000,0.0,0.667000,0.333000,B6,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.067,0.070,0.067,0.070,0.076,0.090,0.108,0.120,0.127,0.133
62,S3_B7_03-16-2021_Right,0.0000,0.000000,0.0,0.778000,0.222000,B7,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.066,0.070,0.065,0.067,0.071,0.080,0.093,0.103,0.109,0.116


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [15]:
temp_hold_path = r"03_16_21_Synthesis_Plus_Spectra_fix"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)
combined_df

,UID,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,Well,Labware,Slot,dppc-ethanol-stock uL,...,900.0nm,910.0nm,920.0nm,930.0nm,940.0nm,950.0nm,960.0nm,970.0nm,980.0nm,990.0nm
0,S2_A1_03-16-2021_Right,0.0001,0.000035,0.0,0.333333,0.666532,A1,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.112,0.113,0.121,0.121,0.140,0.161,0.192,0.207,0.220,0.200
1,S2_A2_03-16-2021_Right,0.0001,0.000035,0.0,0.444444,0.555421,A2,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.070,0.072,0.072,0.078,0.087,0.108,0.134,0.148,0.154,0.158
2,S2_A3_03-16-2021_Right,0.0001,0.000035,0.0,0.555556,0.444309,A3,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.067,0.069,0.068,0.073,0.081,0.098,0.121,0.134,0.141,0.145
3,S2_A4_03-16-2021_Right,0.0001,0.000035,0.0,0.666667,0.333198,A4,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.068,0.071,0.068,0.072,0.078,0.092,0.111,0.123,0.129,0.135
4,S2_A5_03-16-2021_Right,0.0001,0.000035,0.0,0.777778,0.222087,A5,Falcon 48 Well Plate 1500 ÂµL,2,75.000004,...,0.069,0.073,0.067,0.070,0.074,0.084,0.097,0.107,0.114,0.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,S3_B4_03-16-2021_Right,0.0000,0.000000,0.0,0.444000,0.556000,B4,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.121,0.124,0.125,0.132,0.143,0.166,0.194,0.209,0.218,0.222
60,S3_B5_03-16-2021_Right,0.0000,0.000000,0.0,0.556000,0.444000,B5,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.076,0.078,0.075,0.082,0.090,0.110,0.134,0.148,0.154,0.163
61,S3_B6_03-16-2021_Right,0.0000,0.000000,0.0,0.667000,0.333000,B6,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.067,0.070,0.067,0.070,0.076,0.090,0.108,0.120,0.127,0.133
62,S3_B7_03-16-2021_Right,0.0000,0.000000,0.0,0.778000,0.222000,B7,Falcon 48 Well Plate 1500 ÂµL,3,0.000000,...,0.066,0.070,0.065,0.067,0.071,0.080,0.093,0.103,0.109,0.116


In [9]:
from Prepare import CreateSamples

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [ ]:
CreateSamples.team_drive_dict()

In [ ]:
current_members = CreateSamples.file_and_folder_navi('1dXYmxuESNhgmVHntEf8hnZmjK0Br8b64')
CreateSamples.file_and_folder_navi('1CDnoLE32bG0BFhhvCl5JlRFO4I_ga-eJ')

In [ ]:
file_to_upload = temp_hold_path 
folder_to_place_id = '1YXrIIXT3uSFd5-NSpht-1wB1-7hH6Y1w'
name_upload = 'Synthesis and UVVis'

CreateSamples.upload_to_team_drive_folder(folder_to_place_id, file_to_upload, name_upload)